In [ ]:
%pip install -U pip pandas numpy

In [ ]:
import datetime

import pandas as pd
from sqlalchemy import text

from summit_db import *

Example of the expected input for the code:

```
storage_urls,file_name,file_size,md5sum,case_ids,study_uid,series_uid,instance_uid,submission_uid
replicated-data-acr/acrimage/2021/11/2.16.840.1.114274.1818.47455604957111075337316216515213272202/2.16.840.1.114274.1818.54466377751733170579364934456562112131.dcm,2.16.840.1.114274.1818.54466377751733170579364934456562112131.dcm,232592,d282e8db56cf88b5fd774df36efd0f8b,10003752-XgWoDDdRAU66JNviBhIKOA,2.16.840.1.114274.1818.47455604957111075337316216515213272202,2.16.840.1.114274.1818.48898871715409804392517238022035041185,2.16.840.1.114274.1818.54466377751733170579364934456562112131,ACR_20211115
```

* RSNA_20221206_image_manifests (submitted)
* RSNA_20230105_image_manifests (submitting)
* RSNA_20230106_image_manifests
* RSNA_20230117_image_manifests
* RSNA_20230131_image_manifests
* RSNA_20230214_image_manifests (submitted)
* RSNA_20230303_image_manifests (submitted)
* RSNA_20230324_image_manifests (submitted)
* RSNA_20230403_image_manifests (submitted)
* RSNA_20230425_image_manifests

In [ ]:
# df = process_submission("data_example/RSNA_20230303_image_manifests/new_image_manifest_RSNA_20230303.tsv", "RSNA_20230303")
batch_name="RSNA_20221206"
df = process_submission("/home/ubuntu/wd/temp_manifests/new_image_manifest_{}.tsv".format(batch_name), batch_name)


In [ ]:
with eng.begin() as conn:
    df.to_sql("everything", con=conn, schema="public", if_exists="append", index=False)

In [ ]:
sub_df = get_submission([batch_name])

In [ ]:
sub_df[["file_size"]].sum()/(1000**4)

In [ ]:
#record_type	guid	md5	size	authz	urls	file_name	package_contents
#package	dg.MD1R/639703ae-1a20-485d-b710-7bf35bf376df	b0d9dbddeb92d237c357d8d839a7e479	13746408	"[""/programs/Open/projects/R1""]"	s3://open-data-midrc/zip/419639-007644/1.2.826.0.1.3680043.10.474.419639.314881501950920449673698361907/1.2.826.0.1.3680043.10.474.419639.259674532612370694815533893848.zip	419639-007644/1.2.826.0.1.3680043.10.474.419639.314881501950920449673698361907/1.2.826.0.1.3680043.10.474.419639.259674532612370694815533893848.zip	"[{""hashes"": {""md5sum"": ""a72b75baa79e2e7bef4fd0fde2ab4626""}, ""file_name"": ""1.2.826.0.1.3680043.10.474.419639.259674532612370694815533893848/1.2.826.0.1.3680043.10.474.419639.104705416102949685117920506906.dcm"", ""size"": 13746044}]"

In [ ]:
with eng.connect().execution_options(autocommit=True) as conn:
    df2 = pd.read_sql(text('''
    SELECT json_agg(
    json_build_object(
        'hashes', json_build_object('md5sum', md5sum),
        'file_name', series_uid || '/' || file_name,
        'size', file_size
        )
)
FROM everything
where everything.submission_id = 'RSNA_20221206'
group by series_uid;
    '''), con=conn)

In [ ]:
df2.loc[0, "json_agg"]

In [ ]:
df2

* cases
* studies
* series
* instances

* `case_ids`/`study_uid`/`series_uid`/`instance_uid`
* `submission_id`
* `submitted` — Date (YYYY-MM-DD)
* `removed` — Date (YYYY-MM-DD)

In [ ]:
cases = df.loc[:, ["case_ids", "submission_id"]].drop_duplicates()
cases["submitted"] = datetime.date(2023, 5, 12)
cases["removed"] = None

In [ ]:
cases

In [ ]:
with eng.begin() as conn:
    cases.to_sql("cases", con=conn, schema="public", if_exists="append", index=False)

In [ ]:
get_all_cases().loc[get_all_cases()["submitted"] < datetime.date(2023, 8, 31)]